# folium을 이용한 서울시 스타벅스 이디야 입지 시각화
- 참고 : https://wooiljeong.github.io/python/folium/

- 필요한 import 및 from 작업

In [1]:
import folium
import json
import glob
import os
import pandas as pd
import numpy as np

- 가장 최근 "소상공인시장진흥공단_상가(상권)정보 파일"(22년 9월)을 다운로드합니다.
- 링크 : https://www.data.go.kr/data/15083033/fileData.do

- 데이터 불러오기

In [2]:
path = 'data/'
allFiles = glob.glob(os.path.join(path+"*.csv"))

df = pd.DataFrame()
for file in allFiles:

    print(file)

    temp = pd.read_csv(file, engine='python', encoding='utf-8')
    df = pd.concat([df, temp])

data\소상공인시장진흥공단_상가(상권)정보_서울_202209.csv


- 데이터 컬럼 확인하기

In [3]:
print('컬럼 항목 수 :', len(df.columns))
list(df.columns)

컬럼 항목 수 : 39


['상가업소번호',
 '상호명',
 '지점명',
 '상권업종대분류코드',
 '상권업종대분류명',
 '상권업종중분류코드',
 '상권업종중분류명',
 '상권업종소분류코드',
 '상권업종소분류명',
 '표준산업분류코드',
 '표준산업분류명',
 '시도코드',
 '시도명',
 '시군구코드',
 '시군구명',
 '행정동코드',
 '행정동명',
 '법정동코드',
 '법정동명',
 '지번코드',
 '대지구분코드',
 '대지구분명',
 '지번본번지',
 '지번부번지',
 '지번주소',
 '도로명코드',
 '도로명',
 '건물본번지',
 '건물부번지',
 '건물관리번호',
 '건물명',
 '도로명주소',
 '구우편번호',
 '신우편번호',
 '동정보',
 '층정보',
 '호정보',
 '경도',
 '위도']

- 상권업종 간단하게 확인하기

In [4]:
print('='*70)
print('상권업종대분류명', set(df['상권업종대분류명']))
print('='*70)
print('상권업종중분류명', set(df['상권업종중분류명']))
print('='*70)

상권업종대분류명 {'학문/교육', '스포츠', '생활서비스', '소매', '숙박', '부동산', '음식', '관광/여가/오락'}
상권업종중분류명 {'음/식료품소매', '대중목욕탕/휴게', '행사/이벤트', '일식/수산물', '학원-예능취미체육', '기타판매업', '철물/난방/건설자재소매', '평가/개발/관리', '분식', '놀이/여가/취미', '학문교육기타', '패스트푸드', 'PC/오락/당구/볼링등', '분양', '학원-컴퓨터', '책/서적/도서', '사진/광학/정밀기기소매', '운동/경기용품소매', '별식/퓨전요리', '애견/애완/동물', '유아교육', '종합소매점', '법무세무회계', '부동산관련서비스', '예식/의례/관혼상제', '대행업', '유흥주점', '개인서비스', '모텔/여관/여인숙', '경마/경륜/성인오락', '커피점/카페', '세탁/가사서비스', '물품기기대여', '종교용품판매', '취미/오락관련소매', '광고/인쇄', '주택수리', '사무/문구/컴퓨터', '가정/주방/인테리어', '부동산중개', '무도/유흥/가무', '제과제빵떡케익', '유아용품', '화장품소매', '캠프/별장/펜션', '중고품소매/교환', '개인/가정용품수리', '장례/묘지', '시계/귀금속소매', '자동차/자동차용품', '페인트/유리제품소매', '닭/오리요리', '가구소매', '건강/미용식품', '의복의류', '가방/신발/액세서리', '한식', '학원-창업취업취미', '유스호스텔', '호텔/콘도', '양식', '학원-보습교습입시', '이/미용/건강', '기타음식업', '기타서비스업', '중식', '운송/배달/택배', '가전제품소매', '민박/하숙', '연극/영화/극장', '학원기타', '운영관리시설', '주유소/충전소', '실내운동시설', '자동차/이륜차', '예술품/골동품/수석/분재', '부페', '실외운동시설', '도서관/독서실', '의약/의료품소매', '학원-어학', '사진', '음식배달서비스', '스포츠/운동', '학원-음악미술무용', '학원-자격/국가고시', '

- 관련 정보만 추출

In [5]:
dataset = df[['상호명','지점명',
              '상권업종대분류명', '상권업종중분류명',
              '시도명', '시군구명', '행정동명',
              '위도', '경도']]

dataset.head()

상호명  지점명 상권업종대분류명   상권업종중분류명    시도명  시군구명         행정동명         위도  \
0       제중건강원  NaN       소매    건강/미용식품  서울특별시  영등포구         영등포동  37.520613   
1       민속악기사  NaN       소매  취미/오락관련소매  서울특별시   성동구          용답동  37.566857   
2       태평양진주  NaN       소매   시계/귀금속소매  서울특별시   종로구  종로1.2.3.4가동  37.571848   
3  김선희꼼꼼국어교습소  NaN    학문/교육  학원-보습교습입시  서울특별시   송파구         방이1동  37.510967   
4         비지트  NaN       음식         한식  서울특별시   서초구         방배4동  37.488375   

           경도  
0  126.907168  
1  127.049018  
2  126.993530  
3  127.121520  
4  126.991394

- 커피 전문점 데이터 추출
- 여러 상권업소 중 서울시 내 커피 전문점 업소만 추출해 df_coffee를 정의합니다.
- 즉, 시도명이 서울특별시이면서, 상권업종중분류명이 커피점/카페인 경우를 뜻합니다.

In [6]:
dataset[dataset['상호명']=='스타벅스']

상호명       지점명 상권업종대분류명 상권업종중분류명    시도명  시군구명  행정동명         위도  \
1495    스타벅스    동숭로아트점       음식   커피점/카페  서울특별시   종로구   이화동  37.582964   
2516    스타벅스     현대목동점       음식   커피점/카페  서울특별시   양천구   목1동  37.527147   
3535    스타벅스     가로수길점       음식   커피점/카페  서울특별시   강남구   신사동  37.523184   
3614    스타벅스       당산점       음식   커피점/카페  서울특별시  영등포구  당산2동  37.535134   
4595    스타벅스      문래역점       음식   커피점/카페  서울특별시  영등포구   문래동  37.517450   
...      ...       ...      ...      ...    ...   ...   ...        ...   
272352  스타벅스       NaN       음식   커피점/카페  서울특별시   광진구  중곡1동  37.562814   
272724  스타벅스  태릉입구역DT점       음식   커피점/카페  서울특별시   노원구  공릉1동  37.619249   
273252  스타벅스     서울역사점       음식   커피점/카페  서울특별시   용산구   남영동  37.554828   
273791  스타벅스    도산사거리점       음식   커피점/카페  서울특별시   강남구  논현2동  37.519593   
318076  스타벅스     서대문역점       음식   커피점/카페  서울특별시   종로구   교남동  37.566791   

                경도  
1495    127.003887  
2516    126.874682  
3535    127.021629  
3614    126.899952  
4595    126.896479  
...            ...  
272352  127.081329  
272724  127.079298  
273252  126.971712  
273791  127.034097  
318076  126.966238  

[322 rows x 9 columns]

In [7]:
df_coffee = dataset[(dataset['상권업종중분류명']=='커피점/카페')&(dataset['시도명']=='서울특별시')]
df_coffee.index = range(len(df_coffee))
print('서울시 내 커피 전문점 점포 수 :', len(df_coffee))
df_coffee.head()

서울시 내 커피 전문점 점포 수 : 20842


상호명          지점명 상권업종대분류명 상권업종중분류명    시도명 시군구명  행정동명         위도  \
0    커피빈  코리아대학로대명거리점       음식   커피점/카페  서울특별시  종로구   혜화동  37.583149   
1  요거프레소          쌍문점       음식   커피점/카페  서울특별시  도봉구  쌍문2동  37.658728   
2  우성커피숍          NaN       음식   커피점/카페  서울특별시  양천구  신월4동  37.522055   
3   버블베어          NaN       음식   커피점/카페  서울특별시  강서구  방화3동  37.580941   
4    알뤼르          NaN       음식   커피점/카페  서울특별시  강남구  대치4동  37.504378   

           경도  
0  127.000171  
1  127.034746  
2  126.842935  
3  126.813358  
4  127.054731

In [8]:
df_coffee['상호명']

0              커피빈
1            요거프레소
2            우성커피숍
3             버블베어
4              알뤼르
           ...    
20837    공유KYOUYUU
20838         운비차실
20839           풀꽃
20840          댄싱컵
20841         플랫카페
Name: 상호명, Length: 20842, dtype: object

- 스타벅스 데이터 추출

In [9]:
df_seoul_starbucks = df_coffee[df_coffee['상호명'].str.contains('스타벅스')]
df_seoul_starbucks.index = range(len(df_seoul_starbucks))
print('서울시 내 스타벅스 점포 수 :', len(df_seoul_starbucks))
df_seoul_starbucks.head()

서울시 내 스타벅스 점포 수 : 460


상호명      지점명 상권업종대분류명 상권업종중분류명    시도명 시군구명  행정동명         위도  \
0         스타벅스   동숭로아트점       음식   커피점/카페  서울특별시  종로구   이화동  37.582964   
1  스타벅스남부터미널2점  남부터미널2점       음식   커피점/카페  서울특별시  서초구  서초3동  37.484386   
2         스타벅스    현대목동점       음식   커피점/카페  서울특별시  양천구   목1동  37.527147   
3  스타벅스미아사거리역점  미아사거리역점       음식   커피점/카페  서울특별시  강북구   송중동  37.611841   
4         스타벅스    가로수길점       음식   커피점/카페  서울특별시  강남구   신사동  37.523184   

           경도  
0  127.003887  
1  127.014198  
2  126.874682  
3  127.030685  
4  127.021629

- 이디야 데이터 추출

In [10]:
df_seoul_ediya = df_coffee[df_coffee['상호명'].str.contains('이디야')]
df_seoul_ediya.index = range(len(df_seoul_ediya))
print('서울시 내 이디야 점포 수 :', len(df_seoul_ediya))
df_seoul_ediya.head()

서울시 내 이디야 점포 수 : 427


상호명    지점명 상권업종대분류명 상권업종중분류명    시도명  시군구명  행정동명         위도  \
0       이디야커피   신길역점       음식   커피점/카페  서울특별시  영등포구  신길1동  37.515118   
1       이디야커피   라이프점       음식   커피점/카페  서울특별시  영등포구   여의동  37.519544   
2  이디야커피양재AT점  양재AT점       음식   커피점/카페  서울특별시   서초구  양재2동  37.469075   
3       이디야커피    시흥점       음식   커피점/카페  서울특별시   금천구  시흥2동  37.449066   
4       이디야커피  개봉중앙점       음식   커피점/카페  서울특별시   구로구  개봉3동  37.486436   

           경도  
0  126.918062  
1  126.939198  
2  127.040265  
3  126.913646  
4  126.854872

- 메가커피 데이터 추출

In [11]:
df_seoul_mega = df_coffee[df_coffee['상호명'].str.contains('메가커피')]
df_seoul_mega.index = range(len(df_seoul_mega))
print('서울시 내 이디야 점포 수 :', len(df_seoul_mega))
df_seoul_mega.head()

서울시 내 이디야 점포 수 : 87


상호명       지점명 상권업종대분류명 상권업종중분류명    시도명  시군구명  행정동명         위도          경도
0  메가커피     아차산역점       음식   커피점/카페  서울특별시   광진구  중곡4동  37.552647  127.090134
1  메가커피     신설동역점       음식   커피점/카페  서울특별시  동대문구   용신동  37.576200  127.023642
2  메가커피    개봉사거리점       음식   커피점/카페  서울특별시   구로구  개봉1동  37.497108  126.856168
3  메가커피      동대문점       음식   커피점/카페  서울특별시    중구   광희동  37.567824  127.007835
4  메가커피  방학도깨비시장점       음식   커피점/카페  서울특별시   도봉구  방학2동  37.665135  127.036000

- 서울시 구 별 스타벅스 수

In [12]:
starbucks_gu = df_seoul_starbucks.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False)
starbucks_gu = starbucks_gu.reset_index()
starbucks_gu = starbucks_gu.set_index('시군구명')
starbucks_gu

상호명
시군구명     
강남구    76
중구     42
서초구    37
송파구    33
영등포구   28
종로구    27
마포구    26
강서구    16
용산구    16
광진구    16
성북구    15
서대문구   15
강동구    13
양천구    13
노원구    11
관악구    11
금천구    11
동작구    11
구로구    10
동대문구    9
은평구     6
중랑구     6
성동구     5
강북구     5
도봉구     2

- 서울시 구 별 이디야 수

In [13]:
ediya_gu = df_seoul_ediya.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False)
ediya_gu = ediya_gu.reset_index()
ediya_gu = ediya_gu.set_index('시군구명')
ediya_gu

상호명
시군구명     
강서구    35
송파구    29
강남구    26
영등포구   22
관악구    22
마포구    20
금천구    18
은평구    18
동대문구   18
강동구    18
구로구    17
성북구    17
노원구    16
양천구    16
중랑구    14
광진구    13
도봉구    13
중구     13
성동구    13
서초구    12
서대문구   12
동작구    12
강북구    12
용산구    11
종로구    10

- 서울시 구 별 메가커피 수

In [14]:
mega_gu = df_seoul_mega.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False)
mega_gu = mega_gu.reset_index()
mega_gu = mega_gu.set_index('시군구명')
mega_gu

상호명
시군구명     
동작구     9
구로구     7
마포구     6
송파구     5
강서구     5
도봉구     5
양천구     4
은평구     4
영등포구    4
동대문구    4
노원구     4
중구      4
관악구     4
서초구     3
성북구     3
중랑구     3
용산구     2
종로구     2
강남구     2
성동구     2
광진구     2
서대문구    2
금천구     1

- 서울특별시 폴리곤 JSON 파일 불러오기
- https://github.com/PinkWink/DataScience/blob/master/data/02.%20skorea_municipalities_geo_simple.json

folium을 통해 서울시 지도 위에 점포들을 시각화하기 위해서는 지리 정보를 담고 있는 폴리곤 json 파일이 필요합니다.
PinkWink(github)에 접속 후 02.%20skorea_municipalities_geo_simple.json을 Data/ 폴더에 다운로드하여 불러옵니다.

간편 다운로드 방법

skorea_municipalities_geo_simple.json 링크를 우클릭하여 다른 이름으로 링크 저장 합니다.
(출처:PinkWink(github))

In [15]:
geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [16]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)
data_size = len(df_seoul_starbucks)

# 지도 정의
map_starbucks = folium.Map(location=loc, zoom_start=12)
map_starbucks.choropleth(geo_data=geo_str,
              data = starbucks_gu['상호명'],
              columns=[starbucks_gu.index, starbucks_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')


# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                 popup=df_seoul_starbucks.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map_starbucks)

map_starbucks

C:\Users\salix\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


- 서울시 구 별 이디야 분포도 시각화

In [17]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)
data2_size = len(df_seoul_ediya)

# 지도 정의
map_ediya = folium.Map(location=loc, zoom_start=12)
map_ediya.choropleth(geo_data=geo_str,
              data = ediya_gu['상호명'],
              columns=[ediya_gu.index, ediya_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')


# 포인트 마커 추가

for i in range(data2_size):

    folium.Marker(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                 popup=df_seoul_ediya.iloc[i][['지점명']],
                 icon=folium.Icon(color='blue')).add_to(map_ediya)

map_ediya

C:\Users\salix\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


- 서울시 스타벅스 및 이디야 입지 시각화

In [18]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)

data_size = len(df_seoul_starbucks)
data2_size = len(df_seoul_ediya)

# 지도 정의
map = folium.Map(location=loc,
                 tiles = 'Stamen Toner',
                 zoom_start=11)

# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                 popup=df_seoul_starbucks.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map)


for i in range(data2_size):

    folium.Marker(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                 popup=df_seoul_ediya.iloc[i][['지점명']],
                 icon=folium.Icon(color='blue')).add_to(map)


map

- 점포 별 반경 100m 서클 시각화

In [19]:
# 위치 파라미터 설정

# 강남역 좌표
loc = [37.497895, 127.027565] # 위도(N), 경도(E)

data_size = len(df_seoul_starbucks)
data2_size = len(df_seoul_ediya)

# 지도 정의
map = folium.Map(location=loc, zoom_start=12)
map.choropleth(geo_data=geo_str,
              data = starbucks_gu['상호명'],
              columns=[starbucks_gu.index, starbucks_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')

# 포인트 서클 추가

for i in range(data_size):

    folium.Circle(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                  radius = 100,
                  popup = df_seoul_starbucks.iloc[i]['지점명'],
                  color = '#2c9147',fill_color = '#2c9147').add_to(map)

for i in range(data2_size):

    folium.Circle(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                  radius = 100,
                  popup = df_seoul_ediya.iloc[i]['지점명'],
                  color = '#32408c',fill_color = '#32408c').add_to(map)

map

C:\Users\salix\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


- 결론   
점포 별 반경 100m 서클 시각화 결과는 강남역을 중심으로 확대한 지도를 보여주고 있습니다. 점포 마다 반경 100m 지역을 원으로 나타내고 있습니다. 반원을 점포의 직접적인 상권으로 가정하면, 초록색 원인 스타벅스와 파란색 원인 이디야 사이에 직접적인 상권이 겹치는 부분이 심심치 않게 보입니다. 또한, 두 브랜드 모두 특정 도로를 중심으로 분포하고 있는 모습을 확인할 수 있습니다. 시각화 결과만 놓고 보면 두 점포가 매우 가까이에 위치해 있는 것을 알 수 있습니다. 다만, 이는 두 브랜드를 비교한 결과이기때문에 스타벅스와 이디야만 근접해있다고 볼 수는 없다는 한계가 있습니다.

In [20]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)

data_size = len(df_seoul_starbucks)
data2_size = len(df_seoul_ediya)

# 지도 정의
map = folium.Map(location=loc,
                 tiles = 'Stamen Toner',
                 zoom_start=11)


# 지도 정의
map = folium.Map(location=loc, zoom_start=12)
map.choropleth(geo_data=geo_str,
              data = starbucks_gu['상호명'],
              columns=[starbucks_gu.index, starbucks_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')

# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                 popup=df_seoul_starbucks.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map)


for i in range(data2_size):

    folium.Marker(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                 popup=df_seoul_ediya.iloc[i][['지점명']],
                 icon=folium.Icon(color='blue')).add_to(map)


map

C:\Users\salix\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
